In [1]:
!pip install nltk

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 128.6 MB/s eta 0:00:00


In [17]:
import pandas as pd
import json
from tqdm import tqdm
from collections import Counter
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [15]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/featurize/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/featurize/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/featurize/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
with open('./stop_word.txt', 'r', encoding='utf-8') as f:
    stop_words = [line.strip() for line in f.readlines()]
stop_words.extend(stopwords.words('english'))

In [23]:
lemmatizer = WordNetLemmatizer()

In [24]:
df1 = pd.read_csv('./arxiv_output.csv')
df1.head()

,arxiv_id,named_entities
0,1,"[{""entity_group"": ""Method"", ""score"": 0.6655504..."
1,2,"[{""entity_group"": ""Method"", ""score"": 0.7490195..."
2,3,"[{""entity_group"": ""Method"", ""score"": 0.5377213..."
3,4,"[{""entity_group"": ""Task"", ""score"": 0.992919981..."
4,5,"[{""entity_group"": ""Method"", ""score"": 0.8687754..."


In [32]:
with open('./abbreviations_file.txt', 'r', encoding='utf-8') as f:
    abbreviations = dict([w.strip().split(',') for w in f.readlines()])

In [26]:
regex = re.compile(r'[@_!#$%^&*()<>?/\|}{~:]')

In [50]:
datalist = []
entity_group = []
for index,row in tqdm(df1.iterrows(), total=df1.shape[0], desc='创建共现:'):
    try:
        items = []
        for item in json.loads(row['named_entities']):
            if item['entity_group'] != 'Generic':
                words = [lemmatizer.lemmatize(word.lower()) for word in nltk.word_tokenize(item['word'])]
                words = [abbreviations[word] if word in abbreviations else word for word in words]
                item['word'] = ' '.join(words)
                if item['word'] not in stop_words and regex.search(item['word']) is None:
                    items.append(item['word'])
                    entity_group.append([item['word'], item['entity_group']]) 
        items = sorted(items)
        for i in range(0, len(items)):
            for j in range(i, len(items)):
                if(items[i] != items[j]):
                    datalist.append([items[i], items[j]])
    except Exception as e:
        print(e)
        print(index, row['named_entities'])
        break

创建共现:: 100%|██████████| 255297/255297 [06:45<00:00, 630.10it/s]


In [51]:
df_oc = pd.DataFrame(datalist, columns=['word1', 'word2'])
df_oc = df_oc.groupby(['word1', 'word2'])['word1'].count().reset_index(name='count').sort_values(by='count', axis=0, ascending=False)
df_oc.head()

,word1,word2,count
7146816,internet,internet of things,658
4054767,deep neural network,neural network,606
7154103,internet of thing,internet of things,605
1124469,artificial intelligence,machine learning,593
3955394,deep learning,neural network,590


In [57]:
df_group = pd.DataFrame(entity_group, columns=['word', 'group']).groupby(['word', 'group'])['word'].count().reset_index(name='count').sort_values(by=['word','count'], axis=0, ascending=[True,False])
df_group.head()

,word,group,count
3,',OtherScientificTerm,213
1,',Method,60
4,',Task,35
2,',Metric,12
0,',Material,6


In [58]:
word_group_list = []
word = ''
for index,row in tqdm(df_group.iterrows(), total=df_group.shape[0]):
    if(row['word'] != word):
        word_group_list.append([row['word'], row['group']])
    word = row['word']

100%|██████████| 1137200/1137200 [01:14<00:00, 15316.56it/s]


In [59]:
df_word_group_unique = pd.DataFrame(word_group_list, columns=['word', 'group'])

In [60]:
df_new = pd.merge(df_oc, df_word_group_unique, left_on='word1', right_on='word', how='left')
df_new.head()

,word1,word2,count,word,group
0,internet,internet of things,658,internet,Material
1,deep neural network,neural network,606,deep neural network,Method
2,internet of thing,internet of things,605,internet of thing,Task
3,artificial intelligence,machine learning,593,artificial intelligence,Task
4,deep learning,neural network,590,deep learning,Method


In [1]:
df_new = pd.merge(df_new, df_word_group_unique, left_on='word2', right_on='word', how='left')
df_new.head(10)

NameError: name 'pd' is not defined

In [63]:
df_new = df_new.rename(columns={'word1': 'entity1', 'word2': 'entity2', 'count': 'oc_count', 'group_x': 'entity1_group', 'group_y': 'entity2_group'})
df_new.drop(columns=['word_x', 'word_y'], inplace=True)
df_new.head()

,entity1,entity2,oc_count,entity1_group,entity2_group
0,internet,internet of things,658,Material,Material
1,deep neural network,neural network,606,Method,Method
2,internet of thing,internet of things,605,Task,Material
3,artificial intelligence,machine learning,593,Task,Method
4,deep learning,neural network,590,Method,Method


In [64]:
df_new.to_csv('./所有词共现.csv', index=False)

In [4]:
import csv

with open('./所有词共现.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    row_count = sum(1 for row in reader)
    print(f'The CSV file has {row_count} rows.')

The CSV file has 10391014 rows.
